In [1]:
import sys
from importlib import reload
sys.path.append('C:/Users/Antoine/github/MEG_pareidolia/python_scripts/Functions')

from MEG_pareidolia_utils import get_pareidolia_bids, p_values_boolean_1d
import mne # Here we import mne, the package that will contain most of the function that we will use today.
from mne.datasets.brainstorm import bst_raw # It is possible to import functions individually. This is helpful since it
                                            # saves time, memory, and makes the calls to the function easier.
from mne.preprocessing import ICA, create_eog_epochs, create_ecg_epochs
import time
import numpy as np
import scipy.io as sio
from scipy.io import savemat, loadmat
import neurokit2 as nk
from scipy import stats
import matplotlib.pyplot as plt
from neurokit2.complexity.complexity_lempelziv import complexity_lempelziv
import pandas as pd

FOLDERPATH = 'D:/Science/PsychoPy_MEG/BIDS_data'
RUN_LIST = {'pareidolia':['1','2','3','4', '5', '6', '7', '8'], 'RS':['1', '2']}
SUBJ_LIST = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11']
#SUBJ_LIST = ['01']
#SUBJ_LIST = ['00']
TASK_LIST = ['pareidolia']

method = None
baseline = (-2.5, -1.5)
tmin = -1
tmax = 1.5
delay=1
dimension=2
tolerance="default"
##Compute complexity
#https://github.com/neuropsychology/NeuroKit/blob/master/neurokit2/complexity/complexity.py



In [7]:
import biotuner

In [8]:
reload(biotuner)

<module 'biotuner' (namespace)>

In [9]:
from biotuner.harmonic_connectivity import harmonic_connectivity


hc = harmonic_connectivity(sf=None,
        data=epochs_data[0],
        peaks_function="EMD",
        precision=0.1,
        n_harm=10,
        harm_function="mult",
        min_freq=2,
        max_freq=80,
        n_peaks=5)

conn_mat = hc.compute_time_resolved_harm_connectivity(
        sf=1200, nIMFs=5, metric="harmsim", delta_lim=50
    )

(3001, 5)
(3001, 5)
(3001, 5)
(3001, 5)
(3001, 5)
(3001, 5)
(3001, 5)
(3001, 5)
(3001, 5)
(3001, 5)
(3001, 5)
(3001, 5)
(3001, 5)
(3001, 5)
(3001, 5)
(3001, 5)


KeyboardInterrupt: 

In [6]:
import numpy as np
import gc
from biotuner.harmonic_connectivity import harmonic_connectivity
#reload(biotuner.harmonic_connectivity)
for subj in SUBJ_LIST:
    all_conn_mat = []
    for task in TASK_LIST:
        for run in RUN_LIST[task]:
            try:
                epochs_name, epochs_path = get_pareidolia_bids(FOLDERPATH, subj, task, run, stage = 'epo_long', cond=None)
                epochs = mne.read_epochs(epochs_path)
                epochs.pick_types(meg=True, ref_meg=False)
                epochs = epochs.crop(tmin, tmax)
                epochs_data = epochs.get_data()
                
                for trial in range(len(epochs_data)):
                    data = epochs_data[trial]
                    hc = harmonic_connectivity(sf=None,
                                               data=data[:],
                                               peaks_function="EMD",
                                               precision=0.5,
                                               n_harm=10,
                                               harm_function="mult",
                                               min_freq=2,
                                               max_freq=80,
                                               n_peaks=5)

                    conn_mat = hc.compute_time_resolved_harm_connectivity(sf=1200, nIMFs=5, metric="harmsim", delta_lim=50)
                    all_conn_mat.append(conn_mat)

                    # Save connectivity matrices after every 10 trials
                    if (trial+1) % 2 == 0:
                        file_name = f"time_resolved_harmonicity_participant_{subj}_task_{task}_run_{run}_trial_{trial}.npy"
                        np.save(file_name, all_conn_mat)
                        all_conn_mat = [] # Reset list for the next set of 10 trials

                        # Clear memory
                        del conn_mat, hc
                        np.empty(0)  # This will release memory held by numpy
                        gc.collect()  # This will release memory held by unreferenced objects

            except FileNotFoundError:
                pass


C:\Users\Antoine\AppData\Local\Temp\ipykernel_13412\3585016298.py:11: RuntimeWarning: This filename (D:/Science/PsychoPy_MEG/BIDS_data\sub-01\ses-recording\meg\sub-01_ses-recording_task-pareidolia_run-1_epo_long.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(epochs_path)


Computing harmonic connectivity for IMF 1
Computing harmonic connectivity for pair (0, 1)
(3001, 5)
Computing harmonic connectivity for pair (0, 2)
(3001, 5)
Computing harmonic connectivity for pair (0, 3)
(3001, 5)
Computing harmonic connectivity for pair (0, 4)
(3001, 5)
Computing harmonic connectivity for pair (0, 5)
(3001, 5)
Computing harmonic connectivity for pair (0, 6)
(3001, 5)
Computing harmonic connectivity for pair (0, 7)
(3001, 5)
Computing harmonic connectivity for pair (0, 8)
(3001, 5)
Computing harmonic connectivity for pair (0, 9)
(3001, 5)
Computing harmonic connectivity for pair (0, 10)
(3001, 5)
Computing harmonic connectivity for pair (0, 11)
(3001, 5)
Computing harmonic connectivity for pair (0, 12)
(3001, 5)
Computing harmonic connectivity for pair (0, 13)
(3001, 5)
Computing harmonic connectivity for pair (0, 14)
(3001, 5)
Computing harmonic connectivity for pair (0, 15)
(3001, 5)
Computing harmonic connectivity for pair (0, 16)
(3001, 5)
Computing harmonic conn

KeyboardInterrupt: 

In [5]:
test = np.load('time_resolved_harmonicity_participant_01_task_pareidolia_run_1_trial_1.npy', allow_pickle=True)
test.shape

(2, 5, 3001, 10, 10)